# 행정 구역별 공공보건의료기관 분석

- concat(연관x 붙임 ), merge(연관 o 붙임)  
- 공통된 사항 묶어서 새로운 DF 만드는게 목적이지 (전처리의 일환)  
- 파일 열어 보고, 필요한 일부만 확인 -> ex) 주소  
- 특이점 확인 => 전처리해야함    ex) 경남 vs. 경상남도  

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('공공보건의료기관현황.csv', index_col=0, encoding='euc_kr')
print(data.columns)
data.head()

Index(['병원 명', '설립형태', '근거 법령', '관계 행정기관', '관계 공공단체', '심평원 요양기관번호', '종별구분',
       '병상수', '소재지 우편번호', '주소', '홈페이지', '대표전화', 'FAX', '비고'],
      dtype='object')


,병원 명,설립형태,근거 법령,관계 행정기관,관계 공공단체,심평원 요양기관번호,종별구분,병상수,소재지 우편번호,주소,홈페이지,대표전화,FAX,비고
연번,,,,,,,,,,,,,,
1,강원도 재활병원,시도립,강원도재활병원설치및운영에관한조례,강원도,해당없음,32200641,병원,165,24227,강원도 춘천시 충열로 142번길 24-16,www.grh.or.kr,033-248-7700,033-248-7723,NaN
2,강원도 삼척의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100060,종합병원,152,25920,강원도 삼척시 오십천로 418,http://ksmc.or.kr,033-572-1141,033-573-8424,NaN
3,강원도 영월의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100078,종합병원,214,26234,강원도 영월군 영월읍 중앙1로 59,http://www.youngwol.org,033-370-9117,033-370-9137,NaN
4,강원도 원주의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100086,종합병원,237,26448,강원도 원주시 서원대로 387(개운동),www.kwmc.or.kr,033-760-4500,033-761-5121,NaN
5,강원도 강릉의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100159,종합병원,137,25535,강원도 강릉시 경강로 2007(남문동 164-1),http://www.gnmc.or.kr,033-646-6910,033-610-1415,NaN


### ['시도'] 컬럼에서 고유한 값 찾기

In [4]:
# 주소에서 시도, 군구 정보 분리 (-> 새로운 DF 만들기)
addr = pd.DataFrame(data['주소'])
addr = addr['주소'].apply(lambda v:v.split()[:2])컬럼에서 고유한 값 찾기
addr = pd.DataFrame(addr.tolist(), columns=('시도', '군구'))
addr

SyntaxError: invalid syntax (2957555140.py, line 3)

In [ ]:
print(addr['시도'].unique())  # 경산시, 창원시 등 수정할 것 있음
print()
print(addr['시도'].unique)

In [ ]:
addr[addr['시도'] == '창원시']
# '경상남도 창원시' 까지만 필요,
# '의창구,마산합포구3.15대로'는 필요가없지

In [ ]:
addr.iloc[27] = ['경상남도', '창원시']
addr.iloc[31] = ['경상남도', '창원시']
addr.iloc[31]

In [ ]:
# 바꿔야 할 '경산시' 확인
addr[addr['시도']=='경산시']
addr.iloc[47] = ['경상북도', '경산시']
addr.iloc[47]

In [ ]:
# 바꿔야 할 '천안시' 확인
# addr[addr['시도']=='천안시']
addr.iloc[209] = ['충청남도', '천안시']
addr.iloc[210] = ['충청남도', '천안시']
addr.iloc[209]
addr.iloc[210]

In [ ]:
# 현재 수정 필요한 것들 전처리 해 주는 과정 중입니다

In [ ]:
# 
# 시도 이름을 표준 이름으로 변경
addr_aliases = {'경기':'경기도', '경남':'경상남도', '경북':'경상북도',
                '충북':'충청북도', '서울시':'서울특별시', '부산특별시':'부산광역시',
                '대전시':'대전광역시', '충남':'충청남도', '전남':'전라남도', '전북':'전라북도'}

addr['시도'] = addr['시도'].apply(lambda v: addr_aliases.get(v, v))
# dict.get(key[,default]) => 생략가능
# -> key에 해당하는 값이 없으면 default값을 리턴
# -> key에 적어놓은 것을 우선적으로 해서, 원하는 대로 채우고자 함

In [ ]:
print(addr['시도'].unique())
# 확인해보면 정상적으로 다 바뀐 것을 알 수 있음
# unique() 활용!!

In [ ]:
# 군구 이름 확인
print(addr['군구'].unique())

In [ ]:
# '도,시'의 정보만 필요하니까, '아란13길'의 위치 확인 후 수정
addr[addr['군구']=='아란13길']
# 제주도였구나, 해당 DF로 찾든, 웹에서 지도를 찾든

In [ ]:
addr.iloc[75] = ['제주특별자도','제주시']
addr.iloc[75]

In [ ]:
# 제대로 되었는지 재확인
print(addr['군구'].unique())

In [ ]:
# 가로축의 시도, 군구 내용을 합침
addr['시도군구'] = addr.apply(lambda r: r['시도'] + ' ' + r['군구'], axis=1)
addr.head()

In [ ]:
# 'count'컬럼 추가 (for 공공의료 기관의 개수 확인)
addr['count'] = 0
addr.head()

In [ ]:
# 행정구역별 공공보건의료기관의 수 구하기
addr_group = pd.DataFrame(addr.groupby(['시도','군구','시도군구'], as_index=False).count())
# 그룹화 한 것을 인덱스로 사용하지 않겠다는 것 (그냥 count용으로만 쓸거니까)
addr_group.head()

In [ ]:
addr_group = addr_group.set_index('시도군구')
addr_group.head()

# 행정구역별 인구 데이터 준비

In [6]:
# 다운받은 것을 수정한 엑셀 파일임
population = pd.read_excel('행정구역시군구별_성별인구수.xlsx')
population.head()

,행정구역(시군구)별(1),행정구역(시군구)별(2),총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


In [7]:
# 앞에서 전처리한 파일과 동일하게 맞춰주기 위한 작업
# 컬럼의 이름 변경
population = population.rename(columns = {'행정구역(시군구)별(1)':'시도','행정구역(시군구)별(2)':'군구'})
population.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


In [8]:
# 병합에 사용할 공통 컬럼 추가
# ['군구'] 컬럼에서 공백부터 제거
# element는 index로 생각해버리셈
row_count = population.shape[0]
for element in range(0, row_count):
    population['군구'][element] = population['군구'][element].strip()

C:\Users\ahasu\AppData\Local\Temp\ipykernel_12780\3350077026.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['군구'][element] = population['군구'][element].strip()


In [9]:
# ['시도']와 ['군구'] 연결하여 ['시도군구'] 컬럼 추가

# r을 DF로 생각하고, 콜론(:)을 기준으로 나눠 지는 것, 인수 개수 무관
# 인자 : 수식 형태 => 인수가 수식에 전달 => 연산의 결과가 앞에 있는 변수로 전달
# population. 이니까 r이 DF로 들어가는 것
# 변수에 넣든지, 새로운 것을 만들든지는 선택이지 (결국 같긴하지)
# 람다 앞에 있는 객체의 성질을 따른다고 보자 (여기서는 population이 DF니까 Df)
population['시도군구'] = population.apply(lambda r : r['시도'] + ' ' + r['군구'], axis =1)
population.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명),시도군구
0,전국,소계,51847509,25862863,25984646,전국 소계
1,서울특별시,소계,9733509,4745088,4988421,서울특별시 소계
2,서울특별시,종로구,151215,73688,77527,서울특별시 종로구
3,서울특별시,중구,126201,61946,64255,서울특별시 중구
4,서울특별시,용산구,229385,110701,118684,서울특별시 용산구


In [10]:
# ['군구'] 컬럼에서 '소계' 항목 제거
# -> 소계 아닌 것만 넣는 다는 것은 제거하겠다는 것
population = population[population.군구 != '소계']
population
# '소계'가 있던 0,1번 행이 날라감

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명),시도군구
2,서울특별시,종로구,151215,73688,77527,서울특별시 종로구
3,서울특별시,중구,126201,61946,64255,서울특별시 중구
4,서울특별시,용산구,229385,110701,118684,서울특별시 용산구
5,서울특별시,성동구,300410,147020,153390,서울특별시 성동구
6,서울특별시,광진구,351263,170164,181099,서울특별시 광진구
...,...,...,...,...,...,...
273,경상남도,함양군,39555,19058,20497,경상남도 함양군
274,경상남도,거창군,62049,30275,31774,경상남도 거창군
275,경상남도,합천군,45054,21590,23464,경상남도 합천군
277,제주특별자치도,제주시,489202,245389,243813,제주특별자치도 제주시


In [ ]:
# population = population.set_index('시도군구')
# 오류문 확인 => 컬럼에 '시도군구'가 없다고 하는 것 -> 이미 인덱스로 설정되었기 때문에
# 이미 인덱스 한번 설정 되었음
population.head()

# 데이터프레임 병합

- 내부 조인 -> 둘 DF에서 조건에 맞는 행 연결 (공통 사항이 존재)  
- 외부 조인 -> 연관이 없는 2개 데이터를 붙여서 크기만 크게 만듦  

- DataFrame.merge(right, how='inner', left_on=None, right_on=None,
left_index=True, right_index=True)  
  -> 인덱스 가지고만 사용하면 left_on=None, right_on=None (안쓰는 거지)  
  

In [ ]:
# 의료기관 데이터와 행정구역별 인구수 데이터 병합
# (데이터가 잘 기록되어있었다면 앞의 전처리 과정 필요없이 바로 merge를 했으면 됐겠지)
addr_population_merge = pd.merge(addr_group, population, how='inner', left_index=True, right_index=True)
addr_population_merge.head()
# 병합 후 이름이 같은 컬럼에는 _x, _y가 붙음

In [ ]:
# 전처리를 하고 병합을 했던 Df에서 최종적으로 필요한 컬럼을 뽑아서 DF를 만듦
# 필요한 컬럼만 추출해서 새로운 데이터프레임 생성
# -> ['시도_x'], ['군구_x'], ['count'], ['총인구수 (명)']

local_medicalcenter_population = addr_population_merge[['시도_x', '군구_x',
'count', '총인구수 (명)']]
# 컬럼명 변경하여 정리
local_medicalcenter_population = local_medicalcenter_population.rename(columns = {'시도_x':'시도', '군구_x':'군구',
'count': '공공의료기관수', '총인구수 (명)':'인구수'})

local_medicalcenter_population.head()
# 최종적으로 쓸 DF 완성

In [ ]:
# 인구대비 공공의료기관 비율 계산 및 컬럼 추가
# -> div 쓰는 방식 익힐 것
mc_count = local_medicalcenter_population['공공의료기관수']
local_population = local_medicalcenter_population['인구수']
local_medicalcenter_population['인구대비의료기관비율'] = mc_count.div(local_population, axis=0) * 100000

local_medicalcenter_population.head()

In [ ]:
# DF로 바로 그래프 그리기 가능
# '의료기관수' 컬럼으로 그린 것
# -> 지역별 공공의료기관수 현황
# ['공공의료기관수'] 컬럼 내림차순 정렬

import matplotlib.pyplot as plt
import platform

plt.style.use('ggplot')
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='AppleGothic')
    
mc_ratio = local_medicalcenter_population[['공공의료기관수']]
mc_ratio = mc_ratio.sort_values('공공의료기관수', ascending=False)
mc_ratio.plot(kind='bar', rot=90, figsize=(30, 6))


In [ ]:
# DF로 바로 그래프 그리기 가능
# '인구대비의료기관비율' 컬럼으로 그린 것
# -> 인구대비 공공의료기관 비율

import matplotlib.pyplot as plt
import platform

plt.style.use('ggplot')
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='AppleGothic')
           
population_mc_rate = local_medicalcenter_population[['인구대비의료기관비율']]
population_mc_rate = population_mc_rate.sort_values('인구대비의료기관비율',
ascending=False)
population_mc_rate.plot(kind='bar', rot=90, figsize=(30, 6))

In [ ]:
# 필요한 것만 DB에서 갖고와서, 자료를 추려 파일을 만들고, 이를 DF 작업해서 쓰는